In [51]:
from pringles.simulator import Simulator
mySimulator = Simulator(cdpp_bin_path='bin/', user_models_dir='src/')

In [116]:
for atomic in mySimulator.atomic_registry.discovered_atomics:
    if atomic.__name__ == "Server":
        Server = atomic
    elif atomic.__name__ == "AutoScaler":
        AutoScaler = atomic
    elif atomic.__name__ == "ServerQueue":
        ServerQueue = atomic
    elif atomic.__name__ == "Dispatcher":
        Dispatcher = atomic
    elif atomic.__name__ == "Attacker":
        Attacker = atomic
    elif atomic.__name__ == "Generator":
        Generator = atomic
   
print(Server)
print(Dispatcher)
print(ServerQueue)
print(Attacker)
print(Generator)


<class 'pringles.models.models.Server'>
<class 'pringles.models.models.Dispatcher'>
<class 'pringles.models.models.ServerQueue'>
<class 'pringles.models.models.Attacker'>
<class 'pringles.models.models.Generator'>


# Descripción de los modelos
El modelo contará con los atómicos de la sección anterior. A continuación la descripción de cada modelo:

- Server: este es un atómico que cuenta con dos puertos de entrada, "job" por el cual le llegarán los IDs de los jobs a procesar, el cual con una distribución exponencial generará un número aleatorio el cual representa el tiempo que le llevará procesar ese job. Una vez que termine el ta correspondiente, enviará por el puerto "done" al dispatcher que realizó el trabajo. Por el puerto de entrada "powerSignal" recibirá las instrucciones del scaler para apagarse o prenderse, y una vez que lo realize avisará por el puerto de salida "ready". Nota: en caso de el server estar procesando un job en el momento que le llega una señal de apagado, esperará a terminar el job y luego se apagará. Este posee un parámetro, "setupTime", que representa el tiempo en que el server demora en prenderse. El apagado es instantáneo.

- Generator: simplemente un generador aleatorio con distribución normal que representará el tráfico "normal" de la red, que estará conectado con la cola del servidor.

- ServerQueue: representa la cola de mensajes del servidor. Recibe los jobs por el puerto de entrada "in" (que vendrán desde el generador y desde el atacante). Recibe también por el puerto de entrada "emit" las solicitudes del dispatcher para que le envíe un nuevo job para procesar, los cuales enviará por el puerto "out". Además, periódicamente la cola informará al scaler su carga actual, por el puerto de salida "queueLoad". Por último, en caso de que la cola este llena, y llegue un job por el puerto de entrada "in", la cola descartará el job, avisando por el puerto de salida "discarded", que estará conectado al output del sistema.

- Dispatcher: este se encargará de pedirle a la cola que le envíe jobs para procesar siempre y cuando haya servidores libres para procesarlo (por el puerto lo pedirá "requestJob", y los recibirá por el puerto "newJob"), que enviará directamente a cada servidor. Además, eventualmente recibirá notificaciones del autoscaler por el puerto "serverStatus" sobre cambios en los servidores. El mensaje será una tupla [ServerID, Off/free], que representa el cierto servidor paso de estar apagado a prendido, o viceversa.

- Scaler: este se encargará de ir recibiendo periódicamente la información de la carga de la cola, y en función de esto irá decidiendo apagar o prender servidores. Poseerá un umbral superior, a partir del cual si es superado por la carga de la cola, decidirá prender un servidor. Por el contrario, si sobrepasa el umbral inferior, apagará un servidor. También estará conectado directamente a todos los servidores para realizar estas acciones.

- Attacker: este se encargará de realizar los ataques DDoS, es decir, irá enviando jobs al puerto de entrada "in" de la cola. Este leerá de un archivo de texto, en el cual se expresa en cada línea el tiempo que deberá esperar para enviar el siguiente job. Es decir, leerá una línea la cual representa un tiempo t, y se programará para enviar un job con un ta = t. 

Referencia: Los datos utilizado para recrear este archivo fueron tomado del sitio https://www.unb.ca/cic/datasets/ddos-2019.html, correspondiente al dataset del segundo día. Fueron tenidos en cuenta de los primeros 100.000 requests los enviados a la ip 10.42.0.215, identificada como víctima del ataque.

## Configuración del modelo
El sistema contará con 10 servidores que inicialmente estarán encendidos. 
El parámetro 'mean' utilizado en los servidores fue elegido específicamente para que pueda soportar la carga del 
ataque pero no por mucho, es decir, si apagamos un servidor, se perderían jobs. Esto fue así, para que al experimentar con el auto-scaler, se vea reflejada una diferencia.

In [96]:
number_of_servers = 10


# El dispatcher conocerá el estado inicial de cada servidor.
dispatcherDefaultConfig = {
    'numberOfServers': number_of_servers
}
for i in range(number_of_servers):
    dispatcherDefaultConfig['server' + str(i)] = 'free'


serverDefaultConfig = {
    'distribution': 'exponential',
    'mean': 0.005,
    'setupTime': '00:00:00:10'
}

serverOnDefaultConifg = {
    'initialStatus': 'free',
    **serverDefaultConfig
}

serverOffDefaultConifg = {
    'initialStatus': 'off',
    **serverDefaultConfig
}

serverQueueDefaultConfig = {
    'size': 1000,
    'currentSizeFrequency': '00:00:00:10'
}

attackerDefaultConfig = {
    'file': '../../../attack-data/ataque.txt'
}

generatorDefaultConfig = {
    'distribution': 'normal',
    'mean': 0.05,
    'deviation': 0.001,
    'initial': 0,
    'increment': 1
}



In [97]:
from pringles.models.errors import PortNotFoundException

dispatcher = Dispatcher('dispatcher', **dispatcherDefaultConfig)
queue = ServerQueue('queue', **serverQueueDefaultConfig)
# creamos dinámicamente los servers y puertos correspondientes en el scaler y dispatcher
servers = {}
for i in range(number_of_servers):
    server_name = 'server' + str(i)
    servers[i] = Server(server_name, **serverOnDefaultConifg)
    try:
        dispatcher.get_port(server_name)
        scaler.get_port(server_name)
    except PortNotFoundException:
        dispatcher.add_outport(server_name)
        scaler.add_outport(server_name)

attacker = Attacker('attacker', **attackerDefaultConfig)
generator = Generator('generator', **generatorDefaultConfig)

In [98]:
print("--Dispatcher ports")
print("Inport names: ", [port.name for port in dispatcher.inports])
print("Outport names: ", [port.name for port in dispatcher.outports])

print("--Server ports--")
print("Inport names: ", [port.name for port in servers[0].inports])
print("Outport names: ", [port.name for port in servers[0].outports])

print("--ServerQueue ports--")
print("Inport names: ", [port.name for port in queue.inports])
print("Outport names: ", [port.name for port in queue.outports])

print("--Attacker ports--")
print("Inport names: ", [port.name for port in attacker.inports])
print("Outport names: ", [port.name for port in attacker.outports])


print("--Generator ports--")
print("Inport names: ", [port.name for port in generator.inports])
print("Outport names: ", [port.name for port in generator.outports])

--Dispatcher ports
Inport names:  ['newJob', 'jobDone', 'serverStatus']
Outport names:  ['requestJob', 'server0', 'server1', 'server2', 'server3', 'server4', 'server5', 'server6', 'server7', 'server8', 'server9']
--Server ports--
Inport names:  ['job', 'powerSignal']
Outport names:  ['done', 'ready']
--ServerQueue ports--
Inport names:  ['in', 'emit']
Outport names:  ['out', 'discarded', 'queueLoad']
--Attacker ports--
Inport names:  []
Outport names:  ['attack']
--Generator ports--
Inport names:  ['stop']
Outport names:  ['out']


In [99]:
from pringles.models import Coupled

subcomponents = [dispatcher, queue, attacker, generator]
for i in range(number_of_servers):
    subcomponents.append(servers[i])
    
    
top_model = Coupled(name='top', subcomponents=subcomponents)

# adding top inports
top_model.add_outport("queueLoad")
top_model.add_outport('discarded')
top_model.add_outport('jobDone')

# adding couplings between queue and dispatcher
top_model.add_coupling(queue.get_port("out"), dispatcher.get_port("newJob"))
top_model.add_coupling(dispatcher.get_port("requestJob"), queue.get_port("emit"))

# adding coupling between attacker and queue
top_model.add_coupling(attacker.get_port('attack'), queue.get_port('in'))
                                                                   

# adding coupling between queue discarded port and top
top_model.add_coupling(queue.get_port('discarded'), 'discarded')

# adding coupling between queue queueLoad and scaler
top_model.add_coupling(queue.get_port('queueLoad'), 'queueLoad')


# adding couplings between dispatcher, scaler and servers
for i in range(number_of_servers):
    server_name = 'server' + str(i)
    # dispatcher/servers couplings
    top_model.add_coupling(dispatcher.get_port(server_name), servers[i].get_port('job'))
    top_model.add_coupling(servers[i].get_port('done'), dispatcher.get_port('jobDone'))
    
    # top/servers couplings
    top_model.add_coupling(servers[i].get_port('done'), 'jobDone')
    

# adding coupling between generator and queue
top_model.add_coupling(generator.get_port('out'), queue.get_port('in'))


print("Top model created")
top_model

Top model created


# Simulación
A continuación crearemos la simulación con la herramienta pringles. Pero la misma no será ejecutada desde jupyter-notebook, dado que el ser esta una simulación que incurre en un tiempo importante, podrá ocasionar problemas con el ambiente de jupyter. Por el contrario, crearemos con pringles el archivo ".ma".

In [100]:
from pringles.simulator import Simulation, Event
from pringles.utils import VirtualTime

sim_evers = []


a_simulation = Simulation(top_model = top_model, 
                          duration = VirtualTime.of_minutes(10),
                          working_dir='sim_results/experimento-sin-scaler'
                         )

dumped_top_model_path = mySimulator.dump_model_in_file(a_simulation.top_model, a_simulation.output_dir)

# results = mySimulator.run_simulation(a_simulation)

print(dumped_top_model_path)

output_path = dumped_top_model_path[:-9] + 'output'

sim_results/experimento-sin-scaler/2019-10-16-005129-7e17fcffed9741898fcf1f4232cd4433/top_model



Por favor dirigirse al directorio de la simulación creada en 'dumped_top_model_path' y ejecutar la línea de comandos:
```
../../../bin/cd++ -m top_model -l logs/ -t 00:00:10:00 -o output
```


Una vez finalizada la simulación veamos la cantidad de paquetes realizados y descartados.

In [101]:
! cat $output_path | grep -c discarded

0


In [102]:
! cat $output_path | grep -c jobdone

9901


Observermos que no fueron descartados paquetes. Es decir, el stack de 10 servidores todo el tiempo encendidos pudieron soportar la carga del ataque.

## Experimentación con el auto-scaler

Agregaremos ahora a la experimentación el auto-scaler. Agregamos la configuración del auto-scaler pero como los demás componentes quedan con la misma configuración, los volvemos a crear con la misma configuración sin agregarla de nuevo

In [103]:
autoScalerDefaultConfig = {
    'numberOfServers': number_of_servers,
    'exponentialWeight' : 0.6,
    'loadLowerBound': 0.3,
    'loadUpperBound': 0.8,
    'loadUpdatesToBreakIdle': 10
}

for i in range(number_of_servers):
    autoScalerDefaultConfig['server' + str(i)] = 'free'


scaler = AutoScaler('autoscaler', **autoScalerDefaultConfig)

In [104]:
from pringles.models.errors import PortNotFoundException

dispatcher = Dispatcher('dispatcher', **dispatcherDefaultConfig)
queue = ServerQueue('queue', **serverQueueDefaultConfig)
# creamos dinámicamente los servers y puertos correspondientes en el scaler y dispatcher
servers = {}
for i in range(number_of_servers):
    server_name = 'server' + str(i)
    servers[i] = Server(server_name, **serverOnDefaultConifg)
    try:
        dispatcher.get_port(server_name)
        scaler.get_port(server_name)
    except PortNotFoundException:
        dispatcher.add_outport(server_name)
        scaler.add_outport(server_name)

attacker = Attacker('attacker', **attackerDefaultConfig)
generator = Generator('generator', **generatorDefaultConfig)

In [105]:
print("--Scaler ports--")
print("Inport names: ", [port.name for port in scaler.inports])
print("Outport names: ", [port.name for port in scaler.outports])

--Scaler ports--
Inport names:  ['queueLoad', 'serverResponse']
Outport names:  ['serverStatus', 'loadAvg', 'server0', 'server1', 'server2', 'server3', 'server4', 'server5', 'server6', 'server7', 'server8', 'server9']


In [108]:
# tendremos las mismas conexiones pero sumaremos además las del autoscaler

from pringles.models import Coupled

subcomponents = [dispatcher, queue, attacker, generator, scaler]
for i in range(number_of_servers):
    subcomponents.append(servers[i])
    
    
top_model = Coupled(name='top', subcomponents=subcomponents)

# adding top inports
top_model.add_outport("queueLoad")
top_model.add_outport('discarded')
top_model.add_outport('jobDone')
top_model.add_outport('loadAvg')

# adding serverStatus coupling between scaler and dispatcher
top_model.add_coupling(scaler.get_port('serverStatus'), dispatcher.get_port("serverStatus"))

# adding couplings between queue and dispatcher
top_model.add_coupling(queue.get_port("out"), dispatcher.get_port("newJob"))
top_model.add_coupling(dispatcher.get_port("requestJob"), queue.get_port("emit"))

# adding coupling between attacker and queue
top_model.add_coupling(attacker.get_port('attack'), queue.get_port('in'))
                                                                   

# adding coupling between queue discarded port and top
top_model.add_coupling(queue.get_port('discarded'), 'discarded')

# adding coupling between queue queueLoad and scaler
top_model.add_coupling(queue.get_port('queueLoad'), 'queueLoad')


# adding couplings between dispatcher, scaler and servers
for i in range(number_of_servers):
    server_name = 'server' + str(i)
    # dispatcher/servers couplings
    top_model.add_coupling(dispatcher.get_port(server_name), servers[i].get_port('job'))
    top_model.add_coupling(servers[i].get_port('done'), dispatcher.get_port('jobDone'))
    
    # scaler/servers couplings
    top_model.add_coupling(servers[i].get_port('ready'), scaler.get_port('serverResponse'))
    top_model.add_coupling(scaler.get_port(server_name), servers[i].get_port('powerSignal'))
    
    # top/servers couplings
    top_model.add_coupling(servers[i].get_port('done'), 'jobDone')
    

# adding coupling between scaler and top of loadAvg to plot over time
top_model.add_coupling(scaler.get_port('loadAvg'), 'loadAvg')

# adding coupling between queue and scaler, where queueLoad is informed
top_model.add_coupling(queue.get_port('queueLoad'), scaler.get_port('queueLoad'))

# adding coupling between generator and queue
top_model.add_coupling(generator.get_port('out'), queue.get_port('in'))


print("Top model created")
top_model

Top model created


In [109]:
from pringles.simulator import Simulation, Event
from pringles.utils import VirtualTime

sim_evers = []


a_simulation = Simulation(top_model = top_model, 
                          duration = VirtualTime.of_minutes(1),
                          working_dir='sim_results/experimento-con-scaler'
                         )

dumped_top_model_path = mySimulator.dump_model_in_file(a_simulation.top_model, a_simulation.output_dir)

# results = mySimulator.run_simulation(a_simulation)

print(dumped_top_model_path)

output_path = dumped_top_model_path[:-9] + 'output'

sim_results/experimento-con-scaler/2019-10-16-005848-f4bf69cfe9d74558bde0121ac557f7d5/top_model


Por favor, nuevamente, dirigirse al directorio de la simulación creada en 'dumped_top_model_path' y ejecutar la línea de comandos:

```
../../../bin/cd++ -m top_model -l logs/ -t 00:00:10:00 -o output
```


Una vez finalizada la simulación veamos la cantidad de paquetes realizados y descartados.

In [114]:
! cat $output_path | grep -c discarded

296


In [115]:
! cat $output_path | grep -c jobdone

9610


Como podemos observar ahora, fueron descartados 296 paquetes.